In [97]:
import pandas as pd
import numpy as np

In [98]:
def feet_to_cm(height):
    if "--" in height:
        return None
    else:
        height = height.replace('″', '')
        feet, inches = map(int, height.split("′"))
        total_inches = feet * 12 + inches
        cm = total_inches * 2.54
        return cm


def lb_to_kg(weight):
    if "--" in weight:
        return None
    else:
        weight = weight.replace(' lb', '')
        return round(int(weight) * 0.453592, 2)

In [99]:
playersInfo = pd.read_csv('../data/csvFiles/playersInfo.csv')
playersIds = pd.read_csv('../data/csvFiles/playersIds.csv')
playersInfo= playersIds.merge(playersInfo, left_on='Player_ID', right_on='Player_ID')
playersInfo

,Player_ID,Player_Name,Height,Weight,Born,Birthplace,Shoots,Draft,GamePlayed,Goals,Assists,Points,PlusMinus
0,8447400,Wayne Gretzky,6′0″,185 lb,1/26/1961,"Brantford, Ontario",L,Undrafted,1487,894,1963,2857,+520
1,8448208,Jaromir Jagr,6′3″,230 lb,2/15/1972,"Kladno, Czechia",L,"1990, PIT (5th overall), 1st round, 5th pick",1733,766,1155,1921,+322
2,8449573,Mark Messier,6′2″,211 lb,1/18/1961,"Edmonton, Alberta",L,"1979, EDM (48th overall), 3rd round, 7th pick",1756,694,1193,1887,+211
3,8449573,Mark Messier,6′2″,211 lb,1/18/1961,"Edmonton, Alberta",L,"1979, EDM (48th overall), 3rd round, 7th pick",1756,694,1193,1887,+211
4,8448000,Gordie Howe,6′0″,205 lb,3/31/1928,"Floral, Saskatchewan",R,Undrafted,157,68,92,160,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7597,8480221,Drake Rymsha,6′0″,187 lb,8/6/1998,"Huntington Woods, Michigan",R,"2017, LAK (138th overall), 5th round, 14th pick",1,0,0,0,+1
7598,8480201,Maksim Sushko,6′0″,206 lb,2/10/1999,"Drahichyn, Belarus",L,"2017, PHI (107th overall), 4th round, 14th pick",2,0,0,0,-1
7599,8480163,Philip Holm,6′1″,190 lb,12/8/1991,"Stockholm, Sweden",L,Undrafted,1,0,0,0,-2
7600,8480161,Libor Sulak,6′2″,207 lb,3/4/1994,"Pelhrimov, Czechia",L,Undrafted,6,0,0,0,-4


In [100]:
playersInfo['Height'] = playersInfo['Height'].apply(feet_to_cm)
playersInfo['Weight'] = playersInfo['Weight'].apply(lb_to_kg)
playersInfo['Born'] = playersInfo['Born'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})', expand=False)
playersInfo['Born'] = pd.to_datetime(playersInfo['Born'], format='%m/%d/%Y')
playersInfo[['City', 'Country']] = playersInfo['Birthplace'].str.split(', ', 1, expand=True)
playersInfo = playersInfo.drop('Birthplace', axis=1)
draft_data = playersInfo['Draft'].str.extract(r'(\d{4}), (\w{3}) \((\d+)(?:st|nd|rd|th) overall\), (\d+)(?:st|nd|rd|th) round, (\d+)(?:st|nd|rd|th) pick')
draft_data.columns = ['DraftYear', 'DraftTeam', 'DraftOverall', 'DraftRound', 'DraftPick']
playersInfo = pd.concat([playersInfo, draft_data], axis=1)
playersInfo = playersInfo.drop('Draft', axis=1)
playersInfo['DraftYear'] = pd.to_numeric(draft_data['DraftYear'], errors='coerce').fillna(0).astype(int)
playersInfo['DraftOverall'] = pd.to_numeric(draft_data['DraftOverall'], errors='coerce').fillna(0).astype(int)
playersInfo['DraftRound'] = pd.to_numeric(draft_data['DraftRound'], errors='coerce').fillna(0).astype(int)
playersInfo['DraftPick'] = pd.to_numeric(draft_data['DraftPick'], errors='coerce').fillna(0).astype(int)
playersInfo['PlusMinus'] = playersInfo['PlusMinus'].astype(str)
playersInfo['Shoots'] = playersInfo['Shoots'].astype(str)
playersInfo.replace('--', np.nan, inplace=True)
playersInfo

,Player_ID,Player_Name,Height,Weight,Born,Shoots,GamePlayed,Goals,Assists,Points,PlusMinus,City,Country,DraftYear,DraftTeam,DraftOverall,DraftRound,DraftPick
0,8447400,Wayne Gretzky,182.88,83.91,1961-01-26,L,1487,894,1963,2857,+520,Brantford,Ontario,0,NaN,0,0,0
1,8448208,Jaromir Jagr,190.50,104.33,1972-02-15,L,1733,766,1155,1921,+322,Kladno,Czechia,1990,PIT,5,1,5
2,8449573,Mark Messier,187.96,95.71,1961-01-18,L,1756,694,1193,1887,+211,Edmonton,Alberta,1979,EDM,48,3,7
3,8449573,Mark Messier,187.96,95.71,1961-01-18,L,1756,694,1193,1887,+211,Edmonton,Alberta,1979,EDM,48,3,7
4,8448000,Gordie Howe,182.88,92.99,1928-03-31,R,157,68,92,160,NaN,Floral,Saskatchewan,0,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7597,8480221,Drake Rymsha,182.88,84.82,1998-08-06,R,1,0,0,0,+1,Huntington Woods,Michigan,2017,LAK,138,5,14
7598,8480201,Maksim Sushko,182.88,93.44,1999-02-10,L,2,0,0,0,-1,Drahichyn,Belarus,2017,PHI,107,4,14
7599,8480163,Philip Holm,185.42,86.18,1991-12-08,L,1,0,0,0,-2,Stockholm,Sweden,0,NaN,0,0,0
7600,8480161,Libor Sulak,187.96,93.89,1994-03-04,L,6,0,0,0,-4,Pelhrimov,Czechia,0,NaN,0,0,0


In [101]:
playersInfo.dtypes

Player_ID                int64
Player_Name             object
Height                 float64
Weight                 float64
Born            datetime64[ns]
Shoots                  object
GamePlayed               int64
Goals                    int64
Assists                  int64
Points                   int64
PlusMinus               object
City                    object
Country                 object
DraftYear                int64
DraftTeam               object
DraftOverall             int64
DraftRound               int64
DraftPick                int64
dtype: object

In [102]:
playersInfo.to_csv('../data/cleanCsvFiles/playersInfo.csv', index=False)

In [103]:
playersSeasons = pd.read_csv('../data/csvFiles/playersSeasons.csv')
playersSeasons.replace('--', np.nan, inplace=True)
columns_to_convert = ['GamePlayed', 'Goals', 'Assists', 'Points', 'PenaltyInfractionMinutes', 'PowerplayGoals', 'PowerplayPoints', 
                      'ShorthandedGoals', 'ShorthandedPoints', 'GameWinningGoals', 'OvertimeGoals', 'Shots']
for column in columns_to_convert:
    playersSeasons[column] = pd.to_numeric(playersSeasons[column], errors='coerce').fillna(0).astype(int)
playersSeasons['ShootingPercentage'] = pd.to_numeric(playersSeasons['ShootingPercentage'], errors='coerce', downcast='float').fillna(0)
playersSeasons['FaceOffPercentage'] = pd.to_numeric(playersSeasons['FaceOffPercentage'], errors='coerce', downcast='float').fillna(0)
playersSeasons

,Player_ID,Season,Team,GamePlayed,Goals,Assists,Points,PlusMinus,PenaltyInfractionMinutes,PowerplayGoals,PowerplayPoints,ShorthandedGoals,ShorthandedPoints,TimeOnIcePerGame,GameWinningGoals,OvertimeGoals,Shots,ShootingPercentage,FaceOffPercentage
0,8447153,1948-49,Maple Leafs,1,0,0,0,NaN,2,0,0,0,0,NaN,0,0,0,0.0,0.000000
1,8480977,2022-23,Maple Leafs,1,0,0,0,+1,0,0,0,0,0,7:16,0,0,0,0.0,0.000000
2,8475188,2011-12,Sabres,25,1,7,8,-1,15,1,1,0,0,17:50,0,0,23,4.4,0.000000
3,8475188,2013-14,Sabres,12,0,0,0,+1,6,0,0,0,0,17:13,0,0,10,0.0,0.000000
4,8475188,2014-15,Kings,71,2,22,24,+11,52,0,5,0,0,15:53,1,0,74,2.7,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49712,8470171,2013-14,Flyers,11,1,1,2,+1,2,0,0,0,0,15:08,0,0,14,7.1,61.500000
49713,8470171,2013-14,Avalanche,70,7,18,25,+4,43,0,1,1,1,16:19,1,0,110,6.4,42.200001
49714,8470171,2014-15,Avalanche,63,5,10,15,+2,27,0,0,0,0,14:00,1,0,73,6.9,49.000000
49715,8470171,2014-15,Bruins,18,0,3,3,-3,2,0,0,0,0,12:13,0,0,24,0.0,36.400002


In [104]:
playersSeasons.dtypes

Player_ID                     int64
Season                       object
Team                         object
GamePlayed                    int64
Goals                         int64
Assists                       int64
Points                        int64
PlusMinus                    object
PenaltyInfractionMinutes      int64
PowerplayGoals                int64
PowerplayPoints               int64
ShorthandedGoals              int64
ShorthandedPoints             int64
TimeOnIcePerGame             object
GameWinningGoals              int64
OvertimeGoals                 int64
Shots                         int64
ShootingPercentage          float32
FaceOffPercentage           float32
dtype: object

In [105]:
playersSeasons.to_csv('../data/cleanCsvFiles/playersSeasons.csv', index=False)